In [46]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import ast

In [2]:
# to get indices - curl -u elastic:elastic -X GET "http://localhost:9200/_cat/indices?v"
# to check auth - curl -u elastic:elastic -X GET "http://localhost:9200/"
# docker run --name elasticsearch -p 9200:9200 -e "discovery.type=single-node"  -e "ELASTIC_PASSWORD=elastic"  -e "xpack.security.enabled=false"  -e "ES_JAVA_OPTS=-Xms2g -Xmx2g" -v data:/usr/share/elasticsearch/data elasticsearch:8.4.2 

In [30]:
# Initialize Elasticsearch client
es = Elasticsearch(hosts=["http://localhost:9200"], timeout=600, http_auth=('elastic', 'elastic'))

/var/folders/c8/fll6dgp97mnd842vr2qlbjr80000gr/T/ipykernel_66388/3071396293.py:2: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(hosts=["http://localhost:9200"], timeout=600, http_auth=('elastic', 'elastic'))
/var/folders/c8/fll6dgp97mnd842vr2qlbjr80000gr/T/ipykernel_66388/3071396293.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts=["http://localhost:9200"], timeout=600, http_auth=('elastic', 'elastic'))


In [31]:
# Check Cluster Health
es.cluster.health()

ObjectApiResponse({'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0})

In [32]:
index_name = 'es_index'

if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name)
        es.indices.put_settings(index="es_index", body={
            "index": {
                "number_of_replicas": 0
            }
        })


In [33]:
# Check Cluster Health
es.cluster.health()

ObjectApiResponse({'cluster_name': 'docker-cluster', 'status': 'red', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 1, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 0.0})

In [34]:
es.cluster.put_settings(body={
    "transient": {
        "cluster.routing.allocation.enable": "all",
        "cluster.routing.allocation.allow_rebalance": "always",
        "cluster.routing.allocation.disk.threshold_enabled": False
    }
})


ObjectApiResponse({'acknowledged': True, 'persistent': {}, 'transient': {'cluster': {'routing': {'allocation': {'allow_rebalance': 'always', 'disk': {'threshold_enabled': 'false'}, 'enable': 'all'}}}}})

In [36]:
# Check Cluster Health
es.cluster.health()

ObjectApiResponse({'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 1, 'active_shards': 1, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0})

In [9]:
# Verify Connection
es.ping()

True

In [10]:
# Get Cluster Information
es.info()


ObjectApiResponse({'name': 'a8b18fad6465', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ANxuoLsaSuapfr-Z78fnmg', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [37]:
# List All Indices
es.indices.get_alias(index="*")

ObjectApiResponse({'es_index': {'aliases': {}}})

In [12]:
meta_file_path = '/Users/rshankar/Downloads/Projects/deep-learning/amazon-search-recommend/data/Digital_Music_Meta.csv'
digital_music_meta_df = pd.read_csv(meta_file_path)

In [38]:
# List All Indices
es.indices.get_alias(index="*")

ObjectApiResponse({'es_index': {'aliases': {}}})

In [19]:
digital_music_meta_df.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,Digital Music,Baja Marimba Band,4.9,8,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],NaN,[],"{'Date First Available': 'February 28, 2010'}",B000V87RP2,NaN
1,Digital Music,'80s Halloween-All Original Artists & Recordings,5.0,3,[],[],14.98,[{'thumb': 'https://m.media-amazon.com/images/...,[],"Love and Rockets (Artist), Duran Duran (...",[],{'Package Dimensions': '5.55 x 4.97 x 0.54 inc...,B0062F0MJQ,NaN


In [21]:
digital_music_meta_df.isna().sum()

main_category          0
title                  7
average_rating         0
rating_number          0
features               0
description            0
price              30412
images                 0
videos                 0
store               4238
categories             0
details                0
parent_asin            0
bought_together    70537
dtype: int64

In [22]:
digital_music_meta_df.fillna('', inplace=True)

/var/folders/c8/fll6dgp97mnd842vr2qlbjr80000gr/T/ipykernel_66388/2364569514.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  digital_music_meta_df.fillna('', inplace=True)


In [49]:
# Prepare documents for indexing
# Prepare documents for indexing
actions = [
    {
        "_index": index_name,
        "_id": str(i),
        "_source": {
            "title": row['title'],
            "description": row['description'],
            "images": row['images'],
            "average_rating": row['average_rating'],
            "rating_number": row['rating_number'],
            "price": f"${float(row['price']):.2f}" if pd.notna(row['price']) and isinstance(row['price'], (int, float, str)) and str(row['price']).replace('.', '', 1).isdigit() else "Price not available",
            "details": {key: value for key, value in ast.literal_eval(row['details']).items() if key.strip()} or {"Not Available": "Not Available"},
            "main_category": row['main_category'] if pd.notna(row['main_category']) and row['main_category'].strip() else "Not Available"


        }
    }
    for i, row in digital_music_meta_df.iterrows()
]

from elasticsearch.helpers import BulkIndexError

try:
    helpers.bulk(es, actions)
except BulkIndexError as e:
    for error in e.errors:
        print(error)


In [16]:
es.cat.shards(index="es_index", h=["index", "shard", "prirep", "state", "unassigned.reason"])

TextApiResponse('es_index 0 p STARTED \n')

In [67]:
es.cluster.health()

ObjectApiResponse({'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 1, 'active_shards': 1, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0})

In [26]:
es.cluster.put_settings(body={
    "transient": {
        "cluster.routing.allocation.enable": "all",
        "cluster.routing.allocation.allow_rebalance": "always",
        "cluster.routing.allocation.disk.threshold_enabled": False  # Temporarily disable disk thresholds
    }
})


ObjectApiResponse({'acknowledged': True, 'persistent': {}, 'transient': {'cluster': {'routing': {'allocation': {'allow_rebalance': 'always', 'disk': {'threshold_enabled': 'false'}, 'enable': 'all'}}}}})

In [50]:
es.indices.delete(index="es_index", ignore=[400, 404])

/var/folders/c8/fll6dgp97mnd842vr2qlbjr80000gr/T/ipykernel_66388/3972522728.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="es_index", ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [24]:
query = 'data'
body = {
        "query": {
            "bool": {
                "should": [
                    {
                        "match": {
                            "title": query  # Match against the title field
                        }
                    },
                    {
                        "match": {
                            "description": query  # Match against the description field
                        }
                    }
                ]
            }
        }
    }
    
top_n = 5
response = es.search(index=index_name, body=body, size=top_n)
docs = response['hits']['hits']  # List of matched documents

/var/folders/c8/fll6dgp97mnd842vr2qlbjr80000gr/T/ipykernel_66388/1597339106.py:22: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=body, size=top_n)


In [25]:
docs

[{'_index': 'es_index',
  '_id': '16616',
  '_score': 12.917667,
  '_source': {'title': 'Data Lords',
   'description': "['2 CD set includes hi-res download card and lots of other extras']",
   'image': "[{'thumb': 'https://m.media-amazon.com/images/I/51gJniZBoZL._SS40_.jpg', 'large': 'https://m.media-amazon.com/images/I/51gJniZBoZL.jpg', 'variant': 'MAIN', 'hi_res': 'https://m.media-amazon.com/images/I/71faHARZn2L._SL1500_.jpg'}]",
   'rating': 4.6}},
 {'_index': 'es_index',
  '_id': '21404',
  '_score': 11.033821,
  '_source': {'title': 'Test Big Data 15154',
   'description': '[]',
   'image': "[{'thumb': 'https://m.media-amazon.com/images/I/211V6992NML._SS40_.jpg', 'large': 'https://m.media-amazon.com/images/I/211V6992NML.jpg', 'variant': 'MAIN', 'hi_res': None}]",
   'rating': 4.7}},
 {'_index': 'es_index',
  '_id': '25878',
  '_score': 11.033821,
  '_source': {'title': 'Test Big Data 1737',
   'description': '[]',
   'image': "[{'thumb': 'https://m.media-amazon.com/images/I/51Z3H

In [27]:
body = {
        "query": {
            "bool": {
                "should": [
                    {
                        "match": {
                            "title": query  # Match against the title field
                        }
                    },
                    {
                        "match": {
                            "description": query  # Match against the description field
                        }
                    }
                ]
            }
        }
    }
    
response = es.search(index=index_name, body=body, size=top_n)
search_results =  response['hits']['hits']  # List of matched documents
    
# Convert search results to DataFrame
results_df = pd.DataFrame([hit['_source'] for hit in search_results])
results_df

/var/folders/c8/fll6dgp97mnd842vr2qlbjr80000gr/T/ipykernel_66388/3403628480.py:20: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=body, size=top_n)


,title,description,image,rating
0,Data Lords,['2 CD set includes hi-res download card and l...,[{'thumb': 'https://m.media-amazon.com/images/...,4.6
1,Test Big Data 15154,[],[{'thumb': 'https://m.media-amazon.com/images/...,4.7
2,Test Big Data 1737,[],[{'thumb': 'https://m.media-amazon.com/images/...,3.6
3,Sundancer,['15 Tracks. Very early band live recording in...,[{'thumb': 'https://m.media-amazon.com/images/...,5.0
4,ONF Goosebumps 6th Mini Album Dahlia Version C...,['ONF Goosebumps 6th Mini Album Dahlia Version...,[{'thumb': 'https://m.media-amazon.com/images/...,5.0


In [26]:
meta_url = "https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/meta_categories/meta_Digital_Music.jsonl.gz"

filename = 'Digital_Music_Meta.csv'
# Specify sample size and directory for saving
sample_size = 100000
directory = '/Users/rshankar/Downloads/Projects/deep-learning/amazon-search-recommend/data'
bm_pickle_file = 'bm25_index.pkl'

#ES
es_index_name = 'es_index'
query = 'data'

get_elasticsearch_results(query, meta_url, index_name, directory, sample_size)

NameError: name 'get_elasticsearch_results' is not defined